In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\leowa\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\leowa\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
import pandas
import pyspark
import numpy
from pyspark.sql import SparkSession

In [4]:
spark=SparkSession.builder.appName('pyspark practice').getOrCreate()

In [11]:
df_1 = spark.read.csv("C:/mvi/New folder (4)/medical_aid_claims (2).csv",header =True)

In [12]:
df_1.count(),df_1.show()

+--------+--------------------+------+---------+-----------+------------+------------+--------------+-----------+---+--------------------+-----------+-----------------+----------------+--------------------+-----+----+----+----+
|    name|               email|gender| location|   employer|relationship|patient_name|patient_suffix|patient_dob|Age|               cause|Fee Charged|membership_period|number_of_claims|number_of_dependants|label|_c16|_c17|_c18|
+--------+--------------------+------+---------+-----------+------------+------------+--------------+-----------+---+--------------------+-----------+-----------------+----------------+--------------------+-----+----+----+----+
|   Chipi|  ajimmison0@sun.com|female|   Kwekwe|       Vipe| Grandfather|     Mabhena|           789|       1995| 28|               Other|       3798|             1541|               2|                   4|    0|NULL|NULL|NULL|
|   Femba|rsoonhouse1@googl...|female|Marondera|       Mudo|     Husband|     Sithole|  

(7000, None)

In [14]:
df_2 = spark.read.csv("C:/mvi/New folder (4)/best_rows.xls",header=True)

In [23]:
df_2.count(),df_2.show()

+--------+--------------------+------+---------+-----------+------------+------------+--------------+-----------+---+--------------------+-----------+-----------------+----------------+--------------------+-----+
|    name|               email|gender| location|   employer|relationship|patient_name|patient_suffix|patient_dob|Age|               cause|Fee Charged|membership_period|number_of_claims|number_of_dependants|label|
+--------+--------------------+------+---------+-----------+------------+------------+--------------+-----------+---+--------------------+-----------+-----------------+----------------+--------------------+-----+
|   Chipi|  ajimmison0@sun.com|female|   Kwekwe|       Vipe| Grandfather|     Mabhena|           789|       1995| 28|               Other|       3798|             1541|               2|                   4|    0|
|   Femba|rsoonhouse1@googl...|female|Marondera|       Mudo|     Husband|     Sithole|           860|       1992| 31|               Other|      4890

(500, None)

DROP THE COLUMNS

In [20]:
df_1 = df_1.drop('_c16')
df_1 = df_1.drop('_c17')
df_1 = df_1.drop('_c18')

In [25]:
df_1.count(),df_1.columns,df_2.count(),df_2.columns

(7000,
 ['name',
  'email',
  'gender',
  'location',
  'employer',
  'relationship',
  'patient_name',
  'patient_suffix',
  'patient_dob',
  'Age',
  'cause',
  'Fee Charged',
  'membership_period',
  'number_of_claims',
  'number_of_dependants',
  'label'],
 500,
 ['name',
  'email',
  'gender',
  'location',
  'employer',
  'relationship',
  'patient_name',
  'patient_suffix',
  'patient_dob',
  'Age',
  'cause',
  'Fee Charged',
  'membership_period',
  'number_of_claims',
  'number_of_dependants',
  'label'])

ADD ADDITIONAL COLUMNS

from pyspark.sql.functions import lit

df_2= df_2.withColumn("_c16",lit("null"))

df_2= df_2.withColumn("_c17",lit("null"))

df_2= df_2.withColumn("_c18",lit("null"))

MERGE TWO DATA FRAMES

In [26]:
df=df_1.union(df_2)

In [27]:
df.count()

7500

In [28]:
# Create a temporary view for SQL queries
df.createOrReplaceTempView("df_sql")

In [29]:
result = spark.sql ("select * from df_sql where name ='Chipi' ")
result.show()

+-----+--------------------+------+---------+-------------+------------+------------+--------------+-----------+---+--------------------+-----------+-----------------+----------------+--------------------+-----+
| name|               email|gender| location|     employer|relationship|patient_name|patient_suffix|patient_dob|Age|               cause|Fee Charged|membership_period|number_of_claims|number_of_dependants|label|
+-----+--------------------+------+---------+-------------+------------+------------+--------------+-----------+---+--------------------+-----------+-----------------+----------------+--------------------+-----+
|Chipi|  ajimmison0@sun.com|female|   Kwekwe|         Vipe| Grandfather|     Mabhena|           789|       1995| 28|               Other|       3798|             1541|               2|                   4|    0|
|Chipi|ebarajaz1c@godadd...|  male|   Harare|        Mycat|     Brother| Chisa Chisi|           226|       1974| 49|    Accident At Home|      14128|   

ADD NULL VALUES DATAFRAME

In [35]:
df_null = spark.read.csv("C:/mvi/New folder (4)/medical_insurance_nullvalues(null shell).csv",header=True)

In [36]:
df_null.show()

+-----------------+-----+--------------+--------------------+-----------+---------+----+-----------+------------+--------+------+--------------------+----------------+-----------+----+
|membership_period|label|patient_suffix|number_of_dependants|patient_dob| location| Age|Fee Charged|patient_name|    name|gender|               cause|number_of_claims|   employer|c_16|
+-----------------+-----+--------------+--------------------+-----------+---------+----+-----------+------------+--------+------+--------------------+----------------+-----------+----+
|             1541|    0|           789|                   4|       1995|   Kwekwe|NULL|       3798|        NULL|   Chipi|female|               Other|               2|       Vipe|null|
|             8269|    0|           860|                   3|       NULL|Marondera|  31|      48905|     Sithole|   Femba|female|               Other|               4|       Mudo|null|
|             6223|    0|           374|                   2|       1999|  

In [44]:
df_null = df_null.drop('c_16')

In [45]:
from pyspark.sql.functions import count
null_df = df_null.select([count(i) for i in df_null.columns])
null_df.show()

+------------------------+------------+---------------------+---------------------------+------------------+---------------+----------+------------------+-------------------+-----------+-------------+------------+-----------------------+---------------+
|count(membership_period)|count(label)|count(patient_suffix)|count(number_of_dependants)|count(patient_dob)|count(location)|count(Age)|count(Fee Charged)|count(patient_name)|count(name)|count(gender)|count(cause)|count(number_of_claims)|count(employer)|
+------------------------+------------+---------------------+---------------------------+------------------+---------------+----------+------------------+-------------------+-----------+-------------+------------+-----------------------+---------------+
|                    6311|        6310|                 6317|                       6297|              6250|           6267|      6290|              6315|               6339|       6275|         6303|        6381|                   6317| 

NON NULL VALUES

In [43]:
from pyspark.sql.functions import count, col

# Count nulls by counting rows, then subtracting non-null counts
null_df = df_null.select([(count("*") - count(c)).alias(c + "_nulls") for c in df_null.columns])

# Show the result
null_df.show()


+-----------------------+-----------+--------------------+--------------------------+-----------------+--------------+---------+-----------------+------------------+----------+------------+-----------+----------------------+--------------+
|membership_period_nulls|label_nulls|patient_suffix_nulls|number_of_dependants_nulls|patient_dob_nulls|location_nulls|Age_nulls|Fee Charged_nulls|patient_name_nulls|name_nulls|gender_nulls|cause_nulls|number_of_claims_nulls|employer_nulls|
+-----------------------+-----------+--------------------+--------------------------+-----------------+--------------+---------+-----------------+------------------+----------+------------+-----------+----------------------+--------------+
|                    689|        690|                 683|                       703|              750|           733|      710|              685|               661|       725|         697|        619|                   683|           716|
+-----------------------+-----------+---

DROP NULL VALUES

In [ ]:
# Remove rows with any null values in the DataFrame
df_cleaned = df_null.na.drop()

# Show the result without null values
df_cleaned.show()

df_cleaned.count()


+-----------------+-----+--------------+--------------------+-----------+---------+---+-----------+------------+--------+------+--------------------+----------------+------------+
|membership_period|label|patient_suffix|number_of_dependants|patient_dob| location|Age|Fee Charged|patient_name|    name|gender|               cause|number_of_claims|    employer|
+-----------------+-----+--------------+--------------------+-----------+---------+---+-----------+------------+--------+------+--------------------+----------------+------------+
|             8832|    0|           729|                   3|       1978|   Rusape| 45|      34948|      Jembwa|Tichaona|female|Road Traffic Acci...|               6|   Babbleset|
|             2445|    0|           160|                   4|       1974|   Kwekwe| 49|      37298|       Evans|   Gweta|female|               Other|               2|  Brainverse|
|             8690|    1|           715|                   2|       2002|   Harare| 21|      22943| 

1617

AGGREGATION PIVOT

In [56]:
from pyspark.sql import functions as F

# Group by and aggregate with min on 'employer' column after pivoting on 'cause'
df_agg = df_cleaned.groupBy(
    'membership_period', 'label', 'patient_suffix', 'number_of_dependants', 'patient_dob', 'location'
).pivot('gender').agg(F.count('employer'))

# Show the result
df_agg.show()


+-----------------+-----+--------------+--------------------+-----------+--------+------+----+
|membership_period|label|patient_suffix|number_of_dependants|patient_dob|location|female|male|
+-----------------+-----+--------------+--------------------+-----------+--------+------+----+
|             1974|    0|           149|                   3|       1984|  Harare|  NULL|   1|
|             5713|    0|           539|                   4|       2001|Bulawayo|  NULL|   1|
|             5674|    0|           317|                   3|       1988|   Gweru|  NULL|   1|
|             6405|    0|           218|                   2|       1973|  Kadoma|     1|NULL|
|             2164|    0|           420|                   2|       2004|  Gwanda|  NULL|   1|
|             3686|    0|           625|                   4|       2003|   Gweru|  NULL|   1|
|             5401|    0|           676|                   1|       1969| Bindura|  NULL|   1|
|              627|    0|           874|          